In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 398.36it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 473.75it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=3, nWay=4, batchSize=8, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [8]:
from training.models.ProtoFOMAML import ProtoFOMAML

pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'c

losses are tensor([0.6096, 0.5796, 0.5888, 0.5826, 0.6262, 0.6495],
       grad_fn=<NllLossBackward0>) and loss is 3.6363797187805176
losses are tensor([0.6054, 0.5738, 0.5860, 0.5802, 0.6229, 0.6479],
       grad_fn=<NllLossBackward0>) and loss is 3.6161749362945557
losses are tensor([0.6015, 0.5683, 0.5830, 0.5774, 0.6198, 0.6460],
       grad_fn=<NllLossBackward0>) and loss is 3.596072196960449
losses are tensor([0.5978, 0.5631, 0.5798, 0.5745, 0.6169, 0.6440],
       grad_fn=<NllLossBackward0>) and loss is 3.576090097427368
losses are tensor([0.5943, 0.5579, 0.5764, 0.5715, 0.6141, 0.6419],
       grad_fn=<NllLossBackward0>) and loss is 3.5561561584472656


/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


outer loop losses are tensor([0.6710, 0.5962, 0.5564, 0.6502, 0.7491, 0.6551],
       grad_fn=<NllLossBackward0>) and loss is 3.8780927658081055
tensor([[ 2.5041e-05, -5.5547e-05, -1.0506e-04,  ..., -1.1521e-04,
         -3.6804e-05,  1.3879e-04],
        [-6.6618e-05,  1.1888e-04, -1.2935e-04,  ..., -3.6110e-05,
          1.1007e-05, -9.4930e-05],
        [ 1.6269e-04, -2.2004e-04, -2.6352e-05,  ..., -9.8340e-05,
          4.0931e-05, -2.0111e-05],
        ...,
        [-8.0745e-05, -3.5357e-05, -5.3032e-04,  ..., -4.3172e-04,
         -6.0024e-04, -1.2584e-04],
        [ 2.3286e-04, -1.2533e-05, -4.3749e-04,  ..., -3.3882e-04,
         -3.3846e-04,  4.9165e-05],
        [-4.0911e-04,  3.0300e-05, -3.8397e-05,  ...,  1.8611e-04,
         -1.8046e-04, -5.0375e-04]])
tensor([ 1.6043e-04, -1.4020e-04,  3.0469e-05,  2.1808e-04,  4.7921e-05,
        -2.2176e-04, -2.2077e-05,  4.3336e-04, -8.8185e-05,  1.1370e-04,
        -1.0157e-04, -5.7788e-05, -1.4339e-04, -3.3849e-04, -3.3915e-04,
    

losses are tensor([0.4833, 0.4707, 0.4435, 0.4329, 0.4275, 0.5242],
       grad_fn=<NllLossBackward0>) and loss is 2.782099962234497
losses are tensor([0.4675, 0.4544, 0.4269, 0.4130, 0.4042, 0.5087],
       grad_fn=<NllLossBackward0>) and loss is 2.674788475036621
losses are tensor([0.4526, 0.4382, 0.4114, 0.3945, 0.3816, 0.4930],
       grad_fn=<NllLossBackward0>) and loss is 2.5713369846343994
losses are tensor([0.4383, 0.4220, 0.3966, 0.3771, 0.3599, 0.4769],
       grad_fn=<NllLossBackward0>) and loss is 2.4708304405212402
losses are tensor([0.4244, 0.4059, 0.3825, 0.3605, 0.3388, 0.4605],
       grad_fn=<NllLossBackward0>) and loss is 2.372546911239624
outer loop losses are tensor([0.6418, 0.8040, 0.6286, 0.5216, 0.4927, 0.8020],
       grad_fn=<NllLossBackward0>) and loss is 3.8907248973846436
tensor([[ 2.0651e-04,  1.4767e-04,  2.6373e-04,  ..., -5.7923e-05,
         -3.5070e-04,  7.5902e-05],
        [-4.6277e-05,  2.2366e-04,  1.1796e-04,  ..., -3.5344e-05,
          2.3077e-

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'c

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'c

In [5]:
import torch

x = torch.Tensor([1,2,3,4,5,6,7,3,4,5,3])
labels = torch.Tensor([1,2,2,2,1,2,1,1,2,1,1])
print(x[torch.where(labels == 1)[0]])
print(x[torch.where(labels == 1)[0]].mean(dim=0))

tensor([1., 5., 7., 3., 5., 3.])
tensor(4.)


In [6]:
a = [[1,3],[2,4]]
b = [[4,2],[1,3]]

a = [sorted(line) for line in a]
b = [sorted(line) for line in b]
print(sorted(a))
print(sorted(b))
print(sorted(a) == sorted(b))

[[1, 3], [2, 4]]
[[1, 3], [2, 4]]
True


In [7]:
x = torch.Tensor([[1,2,5],[3,3,4]])
b = torch.zeros(x.shape)
print(b)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
